In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant_id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://landing-zone-2@ecomadlssud.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/ecomdata",
extra_configs = configs)

True

In [ ]:
%fs ls '/mnt/ecomdata'

path,name,size,modificationTime
dbfs:/mnt/ecomdata/buyers-raw-2/,buyers-raw-2/,0,1751528978000
dbfs:/mnt/ecomdata/countries-raw-2/,countries-raw-2/,0,1751528990000
dbfs:/mnt/ecomdata/sellers-raw-2/,sellers-raw-2/,0,1751528965000
dbfs:/mnt/ecomdata/users-raw-2/,users-raw-2/,0,1751528947000


In [ ]:
%fs ls '/mnt/ecomdata/buyers-raw-2'  

path,name,size,modificationTime
dbfs:/mnt/ecomdata/buyers-raw-2/Buyers-repartition-by-country.parquet,Buyers-repartition-by-country.parquet,13335,1751722754000


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.appName("EcomDataPipeline").getOrCreate()

In [ ]:
spark

In [ ]:
# Read parquet file from /mnt/ecomdata1/users-raw-2 folder

userDF = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("/mnt/ecomdata/users-raw-2")
        


In [ ]:
userDF.show(5)

+--------------------+----+----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+----------------+-----------+
|      identifierHash|type|   country|language|socialNbFollowers|socialNbFollows|socialProductsLiked|productsListed|productsSold|productsPassRate|productsWished|productsBought|gender|civilityGenderId|civilityTitle|hasAnyApp|hasAndroidApp|hasIosApp|hasProfilePicture|daysSinceLastLogin|seniority|seniorityAsMonths|seniorityAsYears|countryCode|
+--------------------+----+----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+-

In [ ]:
userDF.count()

19783

In [ ]:
userDF.head()

Row(identifierHash='-7279641312655250028', type='user', country='Etats-Unis', language='en', socialNbFollowers='3', socialNbFollows='8', socialProductsLiked='0', productsListed='0', productsSold='0', productsPassRate='0.0', productsWished='0', productsBought='0', gender='F', civilityGenderId='2', civilityTitle='mrs', hasAnyApp='False', hasAndroidApp='False', hasIosApp='False', hasProfilePicture='True', daysSinceLastLogin='709', seniority='3205', seniorityAsMonths='106.83', seniorityAsYears='8.9', countryCode='us')

In [ ]:
# Write data to bronze delta lake

userDF.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/users")


In [ ]:
# Read parquet file from /mnt/ecomdata1/buyers-raw-2/ folder

buyersDF = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("/mnt/ecomdata/buyers-raw-2")

In [ ]:
# Write data to bronze delta lake

buyersDF.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/buyers")

In [ ]:
sellersDF = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("/mnt/ecomdata/sellers-raw-2")

In [ ]:
sellersDF.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/sellers")

In [ ]:
countriesDF = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("/mnt/ecomdata/countries-raw-2")

countriesDF.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/countries")